In [1]:
"""
Источник: https://github.com/NVIDIA/NeMo/blob/main/tutorials/asr/Online_Noise_Augmentation.ipynb
"""

#Установка необходимых пакетов
BRANCH = 'main'
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[asr]
!pip install librosa==0.9.2
!pip install json
!pip install soundfile==0.12.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/NVIDIA/NeMo.git (to revision main) to /tmp/pip-install-4p6yd_ow/nemo-toolkit_89dc9b8b77f341da98f68e6a77e8f59e
  Running command git clone --filter=blob:none --quiet https://github.com/NVIDIA/NeMo.git /tmp/pip-install-4p6yd_ow/nemo-toolkit_89dc9b8b77f341da98f68e6a77e8f59e
  Resolved https://github.com/NVIDIA/NeMo.git to commit 3f8eec42efb7ba26919c86900532dd93d2484655
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.5/109.5 KB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 59.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 84.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 KB 21.9 MB/s eta 0:00:00
     

In [2]:
import json
from nemo.collections.asr.parts.preprocessing import perturb, segment
import glob
import soundfile as sf
import os
import librosa

[NeMo W 2023-04-07 11:35:57 optimizers:54] Apex was not found. Using the lamb or fused_adam optimizer will error out.
[NeMo W 2023-04-07 11:36:06 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at your own risk.


* Перейдите в раздел "Общий доступ" в Google Диске.
* Выберите SPbU_smart-assistant
* Щелкните ее правой кнопкой мыши и выберите Добавить ярлык на диск.
* Появится всплывающее окно, выберите MyDrive и нажмите Добавить ярлык.
* Теперь перейдите в блокнот Google Colab и подключитесь к Google Диску с помощью кнопки


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Загрузка набор данных. Это займет несколько мгновений...
print("******")
if not os.path.exists("./raw"):
  os.makedirs("./raw")

for audio_path in glob.glob("/content/drive/MyDrive/SPbU_smart-assistant/raw/*.*"):
  sample, sr = librosa.core.load(audio_path)
  name = audio_path.split('/')[-1]
  sf.write(f'./raw/{name}', sample, samplerate=sr)
print("Конец.\n******")

In [14]:
#Вспомогательная функция из Nemo
def load_audio(filepath) -> segment.AudioSegment:
    sample_segment = segment.AudioSegment.from_file(filepath, target_sr=sr)
    return sample_segment

In [15]:
#Функция создания манифеста шумов
def write_manifest(filepath, data_dir='.', manifest_name='noise_manifest', duration_max=1e9, duration_stride=1.0):
              
    with open(os.path.join(data_dir, manifest_name + '.json'), 'w') as fout:
        y, sr = librosa.load(filepath)
        duration = librosa.get_duration(y=y, sr=sr)

        offsets = []
        durations = []

        if duration > duration_max:
            current_offset = 0.0

            while current_offset < duration:
                difference = duration - current_offset
                segment_duration = min(duration_max, difference)

                offsets.append(current_offset)
                durations.append(segment_duration)

                current_offset += duration_stride

        else:
            offsets.append(0.0)
            durations.append(duration)


        for duration, offset in zip(durations, offsets):
            metadata = {
                'audio_filepath': filepath,
                'duration': duration,
                'label': 'noise',
                'text': '_',  # for compatibility with ASRAudioText collection
                'offset': offset,
            }

            json.dump(metadata, fout)
            fout.write('\n')
            fout.flush()

        name = noise_path.split('/')[-1]
        print(f"Wrote {len(durations)} segments for filename {name}")
            
    print("Finished preparing manifest !")

In [18]:
if not os.path.exists("./noises"):
  os.makedirs("./noises")

for noise_path in glob.glob("/content/drive/MyDrive/SPbU_smart-assistant/noises/*.*"):
  sample, sr = librosa.core.load(noise_path)
  name = noise_path.split('/')[-1]
  sf.write(f'./noises/{name}', sample, samplerate=sr)

In [ ]:
#Манифесты для всех аудио шумов
if not os.path.exists("./manifests"):
  os.makedirs("./manifests")

for noise_path in glob.glob("./noises/*.*"):
  name = noise_path.split('/')[-1][:-4]
  write_manifest(noise_path, data_dir='./manifests', manifest_name=f'noise_{name}', duration_stride=1.0, duration_max=30.0)

In [20]:
import numpy as np
num = 15 #(умножается на количество оригинальных записей)
count = [i for i in range(num)]
first, second, third = np.array_split(count, 3)

In [22]:
#Здесь используется speedperturburation, вероятности шумов снижены
#Вероятности аугументаций
probas = [0.5, 0.5, 0.5, 1.0]

for i in first:
  #пайплайн аугументаций
  perturbations = [
    perturb.WhiteNoisePerturbation(min_level=-80, max_level=-40),
    perturb.GainPerturbation(min_gain_dbfs=0, max_gain_dbfs=30),
    #при изменении количества аудио шумов заменить 7 на новое количество
    perturb.NoisePerturbation(manifest_path=f'./manifests/noise_{i%7+1}.json',
                              min_snr_db=20, max_snr_db=10, max_gain_db=30.0),
    perturb.SpeedPerturbation(sr, 'kaiser_best', min_speed_rate=0.5, 
                              max_speed_rate=2.0, num_rates=-1)]

  augmentations = list(zip(probas, perturbations))
  audio_augmentations = perturb.AudioAugmentor(augmentations)

  for path in glob.glob("./raw/*.*"):
    sample_segment = load_audio(path)
    audio_augmentations.perturb(sample_segment)
    name = path.split('/')[-1][:-4]

    sf.write(f"./drive/MyDrive/SPbU_smart-assistant/auto/{name}_speed_{i}.wav", 
             sample_segment.samples, samplerate=sr, subtype='PCM_24')

[NeMo I 2023-04-07 11:48:33 collections:193] Dataset loaded with 48 files totalling 0.28 hours
[NeMo I 2023-04-07 11:48:33 collections:194] 0 files were filtered totalling 0.00 hours
[NeMo I 2023-04-07 11:49:15 collections:193] Dataset loaded with 35 files totalling 0.17 hours
[NeMo I 2023-04-07 11:49:15 collections:194] 0 files were filtered totalling 0.00 hours
[NeMo I 2023-04-07 11:49:53 collections:193] Dataset loaded with 43 files totalling 0.23 hours
[NeMo I 2023-04-07 11:49:53 collections:194] 0 files were filtered totalling 0.00 hours
[NeMo I 2023-04-07 11:50:36 collections:193] Dataset loaded with 123 files totalling 0.90 hours
[NeMo I 2023-04-07 11:50:36 collections:194] 0 files were filtered totalling 0.00 hours
[NeMo I 2023-04-07 11:51:16 collections:193] Dataset loaded with 168 files totalling 1.28 hours
[NeMo I 2023-04-07 11:51:16 collections:194] 0 files were filtered totalling 0.00 hours


In [ ]:
#Здесь используется TimeStretchPerturbation, вероятности шумов снижены
#Вероятности аугументаций
probas = [0.5, 0.5, 0.5, 1.0]

for i in second:
  #пайплайн аугументаций
  perturbations = [
    perturb.WhiteNoisePerturbation(min_level=-80, max_level=-40),
    perturb.GainPerturbation(min_gain_dbfs=0, max_gain_dbfs=30),
    #при изменении количества аудио шумов заменить 7 на новое количество
    perturb.NoisePerturbation(manifest_path=f'./manifests/noise_{i%7+1}.json',
                              min_snr_db=20, max_snr_db=10, max_gain_db=30.0),
    perturb.TimeStretchPerturbation(min_speed_rate=0.5, max_speed_rate=2, 
                                    num_rates=-1)]

  augmentations = list(zip(probas, perturbations))
  audio_augmentations = perturb.AudioAugmentor(augmentations)

  for path in glob.glob("./raw/*.*"):
    sample_segment = load_audio(path)
    audio_augmentations.perturb(sample_segment)
    name = path.split('/')[-1][:-4]

    sf.write(f"./drive/MyDrive/SPbU_smart-assistant/auto/{name}_stretch_{i}.wav", 
             sample_segment.samples, samplerate=sr, subtype='PCM_24')

In [25]:
#Здесь не используются временные аугументации, вероятности шумов 1
#Вероятности аугументаций
probas = [1.0, 1.0, 1.0]

for i in third:
  #пайплайн аугументаций
  perturbations = [
    perturb.WhiteNoisePerturbation(min_level=-60, max_level=-35),
    perturb.GainPerturbation(min_gain_dbfs=0, max_gain_dbfs=30),
    #при изменении количества аудио шумов заменить 7 на новое количество
    perturb.NoisePerturbation(manifest_path=f'./manifests/noise_{i%7+1}.json',
                              min_snr_db=20, max_snr_db=10, max_gain_db=30.0)]

  augmentations = list(zip(probas, perturbations))
  audio_augmentations = perturb.AudioAugmentor(augmentations)

  for path in glob.glob("./raw/*.*"):
    sample_segment = load_audio(path)
    audio_augmentations.perturb(sample_segment)
    name = path.split('/')[-1][:-4]

    sf.write(f"./drive/MyDrive/SPbU_smart-assistant/auto/{name}_noise_{i}.wav", 
             sample_segment.samples, samplerate=sr, subtype='PCM_24')

[NeMo I 2023-04-07 11:53:34 collections:193] Dataset loaded with 123 files totalling 0.90 hours
[NeMo I 2023-04-07 11:53:34 collections:194] 0 files were filtered totalling 0.00 hours
[NeMo I 2023-04-07 11:53:37 collections:193] Dataset loaded with 168 files totalling 1.28 hours
[NeMo I 2023-04-07 11:53:37 collections:194] 0 files were filtered totalling 0.00 hours
[NeMo I 2023-04-07 11:53:41 collections:193] Dataset loaded with 94 files totalling 0.66 hours
[NeMo I 2023-04-07 11:53:41 collections:194] 0 files were filtered totalling 0.00 hours
[NeMo I 2023-04-07 11:53:44 collections:193] Dataset loaded with 103 files totalling 0.73 hours
[NeMo I 2023-04-07 11:53:44 collections:194] 0 files were filtered totalling 0.00 hours
[NeMo I 2023-04-07 11:53:47 collections:193] Dataset loaded with 48 files totalling 0.28 hours
[NeMo I 2023-04-07 11:53:47 collections:194] 0 files were filtered totalling 0.00 hours
